In [27]:
import pandas as pd
import string
import unicodedata
import sys
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
import sklearn.model_selection as cv
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
import tensorflow as tf
plt.style.use("fivethirtyeight")
%matplotlib inline
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [28]:
smileyfaces = [':-)', ':)', ':D', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)']
sadfaces = ['>:[', ':-(', ':(', ':-c', ':c', ':-<', ':<', ':-[', ':[', ':{', '=(','=[', 'D:']
angryfaces = ['>:(', '(╯°□°)╯︵ ┻━┻']
cryingfaces = [":’-(", ":’("]
skepticalfaces = ['>:', '>:/', ':-/', '=/',':L', '=L', ':S', '>.<']
noexpressionfaces = [':|', ':-|', '(｀・ω・´)']
surprisedfaces = ['>:O', ':-O', ':O', ':-o', ':o', '8O', 'O_O', 'o-o', 'O_o', 'o_O', 'o_o', 'O-O']

In [29]:
def cleanText(wordSeries):
    def remove_punctuation(x):
        for char in string.punctuation:
            x = x.replace(char, ' ')
        return x
    for smile in smileyfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(smile, ' smileyface '))
    for sad in sadfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(sad,' sadface '))
    for angry in angryfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(angry, ' angryface '))
    for cry in cryingfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(cry, ' cryingface '))
    for skeptical in skepticalfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(skeptical, ' skepticalface '))
    for noexp in noexpressionfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(noexp, ' noexpressionfaces '))
    for surprised in surprisedfaces:
        wordSeries = wordSeries.apply(lambda x: x.replace(surprised, ' surprisedface '))
    wordSeries = wordSeries.apply(lambda x: x.replace('...', ' dotdotdot '))
    wordSeries = wordSeries.apply(lambda x: x.replace('!', ' exclamatory '))
    wordSeries = wordSeries.apply(lambda x: remove_punctuation(x))
    wordSeries = wordSeries.apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
    wordSeries = wordSeries.apply(lambda x: x.lower())
    #wordSeries = wordSeries.apply(lambda x: x.replace('http', ' '))
    wordSeries = wordSeries.apply(lambda x: ' '.join( [w for w in x.split() if len(w)>1] ))
    return wordSeries

In [30]:
def tokenize(documents):
    documents = cleanText(documents)
    docs = [word_tokenize(content) for content in documents]
    stopwords_=set(stopwords.words('english'))
    def filter_tokens(sent):
        return([w for w in sent if not w in stopwords_])
    docs=list(map(filter_tokens,docs))
    lemmatizer = WordNetLemmatizer()
    docs_lemma = [[lemmatizer.lemmatize(word) for word in words] for words in docs]
    return docs_lemma

In [31]:
def createTFIDF(data, contentCol):
    data['Tokens'] = tokenize(data[contentCol])
    data['Tokens'] = data['Tokens'].apply(lambda x: ' '.join(x))
    corpus = [row for row in data['Tokens']]
    tfidf = TfidfVectorizer()
    document_tfidf_matrix = tfidf.fit_transform(corpus).toarray()
    document_tfidf_matrix = document_tfidf_matrix[:, :, None]
    return tfidf, document_tfidf_matrix

In [32]:
#dataPN = pd.read_csv('text_polarity.csv', encoding = "ISO-8859-1", header=None)
dataS = pd.read_csv('text_emotion.csv')

In [33]:
dataS['Clean'] = cleanText(dataS['content'])

In [34]:
dataS['BetterClean'] = tokenize(dataS['Clean'])
dataS['BetterClean'] = dataS['BetterClean'].apply(lambda x: " ".join(x))

In [35]:
data = dataS[['sentiment','BetterClean']]

In [54]:
tokenizer = Tokenizer(num_words = 200, split = " ")
tokenizer.fit_on_texts(data['BetterClean'].values)

textVector = tokenizer.texts_to_sequences(data['BetterClean'].values)
textVector = pad_sequences(textVector)

In [55]:
model = Sequential()
model.add(Embedding(200, 256, input_length=textVector.shape[1]))
model.add(Dropout(0.1))
model.add(LSTM(256, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))
model.add(LSTM(256, return_sequences=False, dropout=0.1, recurrent_dropout=0.1))
model.add(Dense(13, activation="softmax"))

In [56]:
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 76, 256)           51200     
_________________________________________________________________
dropout_4 (Dropout)          (None, 76, 256)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 76, 256)           525312    
_________________________________________________________________
lstm_7 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_3 (Dense)              (None, 13)                3341      
Total params: 1,105,165
Trainable params: 1,105,165
Non-trainable params: 0
_________________________________________________________________


In [57]:
y = pd.get_dummies(data['sentiment']).values

In [49]:
textVector[1]

(77,)

In [58]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(textVector, y, test_size=0.9)

In [ ]:
batch_size = 32
epochs = 8
model.fit(Xtrain, Ytrain, epochs=20, batch_size=32, verbose=1)

Train on 4000 samples
Epoch 1/20
4000/4000 [==============================] - 206s 51ms/sample - loss: 2.1516 - acc: 0.2285
Epoch 2/20
4000/4000 [==============================] - 201s 50ms/sample - loss: 2.0317 - acc: 0.2797
Epoch 3/20
4000/4000 [==============================] - 201s 50ms/sample - loss: 1.9713 - acc: 0.3187
Epoch 4/20
3200/4000 [=======================>......] - ETA: 40s - loss: 1.9439 - acc: 0.3300

In [42]:
preds = model.predict(Xtest)

In [43]:
from sklearn.metrics import accuracy_score

In [21]:
Ytest[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=uint8)

In [20]:
preds[0]

array([1.7120372e-04, 4.9815705e-04, 8.5358118e-04, 4.0275161e-04,
       1.4158637e-04, 1.8266856e-04, 4.9179975e-02, 1.8484649e-04,
       3.9006866e-04, 2.3025502e-03, 2.8707633e-02, 1.7433617e-02,
       8.9955139e-01], dtype=float32)

In [53]:
preds[0]

array([0.00148756, 0.00314884, 0.06418547, 0.02302845, 0.02931107,
       0.0526333 , 0.00305231, 0.00406246, 0.6811672 , 0.02787842,
       0.00859896, 0.04841706, 0.05302891], dtype=float32)

In [50]:
np.argmax(preds[0])

8

In [44]:
nonprobPreds = (preds == preds.max(axis=1)[:,None]).astype(int)

In [45]:
accuracy_score(Ytest, nonprobPreds)

0.27125